In [1]:
import os
import re
import sqlite3 as sql
import emoji
import datetime
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

plt.style.use('fivethirtyeight')
#%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
if not os.path.exists('data/fivethirtyeight_tweets.csv'):
    with open('data/fivethirtyeight_tweets.csv', 'w'):
        for ii in range(1,12):
            print('Accessing file '+str(ii)+' of 11...')
            if ii == 1:
                ira_tweets_538 = pd.read_csv('https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_'+str(ii)+'.csv')
            else:
                more = pd.read_csv('https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_'+str(ii)+'.csv')
                ira_tweets_538 = pd.concat([ira_tweets, more])
        ira_tweets_538.to_csv('data/fivethirtyeight_tweets.csv')
        print('Done.')
        pass
else:
    print('Opening existing data file...')
    ira_tweets_538 = pd.read_csv('data/fivethirtyeight_tweets.csv')
    print('Done.')

Opening existing data file...
Done.


In [3]:
ira_tweets_538.head(50)

,Unnamed: 0,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,...,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id,article_url,tco1_step1,tco2_step1,tco3_step1
0,0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,...,Right,0,RightTroll,0,905874659358453760,914580356430536707,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914580356430...,NaN,NaN
1,1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,...,Right,0,RightTroll,0,905874659358453760,914621840496189440,http://twitter.com/905874659358453760/statuses...,https://twitter.com/damienwoody/status/9145685...,NaN,NaN
2,2,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,...,Right,1,RightTroll,0,905874659358453760,914623490375979008,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/913231923715...,NaN,NaN
3,3,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,...,Right,0,RightTroll,0,905874659358453760,914639143690555392,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914639143690...,NaN,NaN
4,4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,...,Right,1,RightTroll,0,905874659358453760,914312219952861184,http://twitter.com/905874659358453760/statuses...,https://twitter.com/realDonaldTrump/status/914...,NaN,NaN
5,5,906000000000000000,10_GOP,"Dan Bongino: ""Nobody trolls liberals better th...",Unknown,English,10/1/2017 2:47,10/1/2017 2:47,1050,9644,...,Right,0,RightTroll,0,905874659358453760,914320835325853696,http://twitter.com/905874659358453760/statuses...,https://twitter.com/FoxNews/status/91423949678...,NaN,NaN
6,6,906000000000000000,10_GOP,🐝🐝🐝 https://t.co/MorL3AQW0z,Unknown,English,10/1/2017 2:48,10/1/2017 2:48,1050,9644,...,Right,1,RightTroll,0,905874659358453760,914321156466933760,http://twitter.com/905874659358453760/statuses...,https://twitter.com/Cernovich/status/914314644...,NaN,NaN
7,7,906000000000000000,10_GOP,'@SenatorMenendez @CarmenYulinCruz Doesn't mat...,Unknown,English,10/1/2017 2:52,10/1/2017 2:53,1050,9644,...,Right,0,RightTroll,0,905874659358453760,914322215537119234,http://twitter.com/905874659358453760/statuses...,NaN,NaN,NaN
8,8,906000000000000000,10_GOP,"As much as I hate promoting CNN article, here ...",Unknown,English,10/1/2017 3:47,10/1/2017 3:47,1050,9646,...,Right,0,RightTroll,0,905874659358453760,914335818503933957,http://twitter.com/905874659358453760/statuses...,http://www.cnn.com/2017/09/27/us/puerto-rico-a...,NaN,NaN
9,9,906000000000000000,10_GOP,After the 'genocide' remark from San Juan Mayo...,Unknown,English,10/1/2017 3:51,10/1/2017 3:51,1050,9646,...,Right,0,RightTroll,0,905874659358453760,914336862730375170,http://twitter.com/905874659358453760/statuses...,NaN,NaN,NaN


In [4]:
ira_tweets_538.columns

Index(['Unnamed: 0', 'external_author_id', 'author', 'content', 'region',
       'language', 'publish_date', 'harvested_date', 'following', 'followers',
       'updates', 'post_type', 'account_type', 'retweet', 'account_category',
       'new_june_2018', 'alt_external_id', 'tweet_id', 'article_url',
       'tco1_step1', 'tco2_step1', 'tco3_step1'],
      dtype='object')

In [5]:
ira_tweets_538.content.head()

0    "We have a sitting Democrat US Senator on tria...
1    Marshawn Lynch arrives to game in anti-Trump s...
2    Daughter of fallen Navy Sailor delivers powerf...
3    JUST IN: President Trump dedicates Presidents ...
4    19,000 RESPECTING our National Anthem! #StandF...
Name: content, dtype: object

Drop columns that I will not use

In [6]:
ira_tweets_538 = ira_tweets_538.drop(['region','Unnamed: 0','article_url','tco1_step1','tco2_step1','tco3_step1'], axis=1)

In [7]:
ira_tweets_538.head()

,external_author_id,author,content,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id
0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,NaN,Right,0,RightTroll,0,905874659358453760,914580356430536707
1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,NaN,Right,0,RightTroll,0,905874659358453760,914621840496189440
2,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,RETWEET,Right,1,RightTroll,0,905874659358453760,914623490375979008
3,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents ...,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,NaN,Right,0,RightTroll,0,905874659358453760,914639143690555392
4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,RETWEET,Right,1,RightTroll,0,905874659358453760,914312219952861184


Remove website urls

In [8]:
ira_tweets_538['content'] = ira_tweets_538.content.str.replace('((www\.[^\s]+)|(http?://[^\s]+)|(https?://[^\s]+))','')

Select only tweets in English

In [9]:
eng_tweets = ira_tweets_538[ira_tweets_538.language == 'English']

In [10]:
eng_tweets = eng_tweets[eng_tweets.account_category != 'NonEnglish']

Remove emojis

In [11]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)

eng_tweets.content = eng_tweets.content.apply(lambda x: emoji_pattern.sub(r'', str(x)))

Add two new columns: list of hashtags in tweet, list of @ mentions

In [12]:
'''
#.*?     -- non-greedy match for words starting with #
(?=\s|$) -- look ahead for the end of the word
'''
eng_tweets['hashtags'] = eng_tweets.content.str.findall(r'#.*?(?=\s|$)')
eng_tweets['mentions'] = eng_tweets.content.str.findall(r'@.*?(?=\s|$)')

Remove punctuation

In [14]:
eng_tweets.content = eng_tweets.content.apply(lambda x: re.sub('[^A-Za-z0-9\' ]+', '', str(x)))
#eng_tweets.content = eng_tweets.content.apply(lambda x: re.sub('[,.!$%&*?;:@#~-+=/]+', '', str(x)))

Split CamelCase words for improved entity recognition

In [21]:
eng_tweets.content = eng_tweets.content.apply(lambda x: re.sub("([a-z])([A-Z])","\g<1> \g<2>", str(x)))

Remove @mentions

In [22]:
#eng_tweets.content = eng_tweets.content.str.replace('@[^\s]+','')

Remove hashtags

In [23]:
#eng_tweets.content = eng_tweets.content.str.replace('#[^\s]+','')

Remove RTs

In [24]:
eng_tweets.content = eng_tweets.content.str.replace('RT','')

Remove apostrophes

In [25]:
#eng_tweets.content = eng_tweets.content.str.replace("\'",'')

Remove 'amp'

In [26]:
eng_tweets.content = eng_tweets.content.str.replace(' amp ',' ')

Finally, remove any extra whitespace

In [27]:
eng_tweets.content = eng_tweets.content.apply(lambda x: re.sub('[\s]+', ' ', x))

Now split tweets into original content and retweets

In [28]:
#user_rt = eng_tweets[eng_tweets.retweet==1]
#user_rt = {}
#for user in eng_tweets.author.unique():
#    tweets = list(eng_tweets.content[(eng_tweets.author==user) & (eng_tweets.retweet==1)].values)
#    user_rt[user] = tweets
#user_rt

In [29]:
#user_original = eng_tweets[eng_tweets.retweet==0]
#user_original = {}
#for user in eng_tweets.author.unique():
#    tweets = list(eng_tweets.content[(eng_tweets.author==user) & (eng_tweets.retweet==0)].values)
#    user_original[user] = tweets
#user_original

In [30]:
#user_original_df = pd.DataFrame()
#user_rt_df = pd.DataFrame()

#user_original_df = user_original_df.append(user_original, ignore_index=True).T
#user_original_df.index.name = 'author'
#user_original_df.columns = ['content']
#user_original_df['content'] = user_original_df['content'].astype('str')
#user_original_df = user_original_df.join(eng_tweets.author)

#user_rt_df = user_rt_df.append(user_rt, ignore_index=True).T
#user_rt_df.index.name = 'author'
#user_rt_df.columns = ['content']
#user_rt_df['content'] = user_rt_df['content'].astype('str')
#user_rt_df = user_rt_df.join(eng_tweets.account_category, on='author')

user_original = eng_tweets[eng_tweets.retweet==0]
user_original = user_original.groupby('author',as_index=False).agg({'account_category': list, 'content': sum})
user_original.account_category = user_original.account_category.apply(lambda x: x[0])

user_rt = eng_tweets[eng_tweets.retweet==1]
user_rt = user_rt.groupby('author',as_index=False).agg({'account_category': list, 'content': sum})
user_rt.account_category = user_rt.account_category.apply(lambda x: x[0])
user_rt.head()

,author,account_category,content
0,10_GOP,RightTroll,Daughter of fallen Navy Sailor delivers powerf...
1,1D_NICOLE_,Fearmonger,Food Poisoning is not a joke Walmart Koch Farm...
2,1ERIK_LEE,RightTroll,Why is someone even against the petition I'll ...
3,4EVER_SUSAN,RightTroll,Raiders defense playing hungry Bending and not...
4,4MYSQUAD,LeftTroll,'politicalseason Maat Justice Injustice in Lou...


Save these dataframes to SQLITE databases

In [31]:
#if not os.path.isfile('../data/user_original.db'):
#    with sql.connect('../data/user_original.db') as conn:
#        user_original_df.to_sql('user_original', conn, index_label='author')
        
#if not os.path.isfile('../data/user_rt.db'):
#    with sql.connect('../data/user_rt.db') as conn:
#        user_rt_df.to_sql('user_rt', conn, index_label='author')

Clean up hashtag and mentions columns to save to db

In [32]:
#for ht_list in eng_tweets.hashtags:
    #print(ht_list[0])

In [33]:
#eng_tweets.hashtags = eng_tweets.hashtags.astype(str).str.replace('#','')
#eng_tweets.hashtags = eng_tweets.hashtags.astype(str).str.replace('.','')

In [34]:
#eng_tweets.hashtags = eng_tweets.hashtags.apply(lambda x: emoji_pattern.sub(r'', str(x)))

In [35]:
#eng_tweets.hashtags = eng_tweets.hashtags.astype(object)

In [36]:
#eng_tweets.mentions = eng_tweets.mentions.astype(str).str.replace('@','')

In [37]:
#eng_tweets.mentions = eng_tweets.mentions.astype(object)

In [38]:
eng_tweets.head()

,external_author_id,author,content,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id,hashtags,mentions
0,906000000000000000,10_GOP,We have a sitting Democrat US Senator on trial...,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,NaN,Right,0,RightTroll,0,905874659358453760,914580356430536707,[],[@nedryun]
1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti Trump s...,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,NaN,Right,0,RightTroll,0,905874659358453760,914621840496189440,[],[]
2,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,RETWEET,Right,1,RightTroll,0,905874659358453760,914623490375979008,[#BoycottNFL],[]
3,906000000000000000,10_GOP,JUST IN President Trump dedicates Presidents C...,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,NaN,Right,0,RightTroll,0,905874659358453760,914639143690555392,[],[]
4,906000000000000000,10_GOP,19000 RESPECTING our National Anthem Stand For...,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,RETWEET,Right,1,RightTroll,0,905874659358453760,914312219952861184,[#StandForOurAnthem],[]


In [39]:
eng_tweets.to_pickle('../data/eng_tweets.p')
user_original.to_pickle('../data/user_original.p')
user_rt.to_pickle('../data/user_rt.p')

In [92]:
#if not os.path.isfile('../data/eng_tweets.db'):
#with sql.connect('../data/eng_tweets.db') as conn:
#    eng_tweets.to_sql('eng_tweets', conn, index=None)